In [1]:
import numpy as np
import cv2
import dlib
from utils import load_face_landmark_model, load_fr_model, load_embeddings_model, load_face_detector, load_inv_label_dictionary

C:\Users\windo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
face_landmarks_model = load_face_landmark_model()
fr_model = load_fr_model()
face_detector = load_face_detector()
font = cv2.FONT_HERSHEY_SIMPLEX
svm = load_embeddings_model(model='svm')
inv_label_dictionary = load_inv_label_dictionary()

# Live Detection

In [3]:
cam = cv2.VideoCapture(0)
while True:
    ret, img = cam.read()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    faces = face_detector(gray, 1)
    try:
        for face_rect in faces:
            x, y, w, h = face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()
            cv2.rectangle(rgb,(x, y),(w, h),(0,255,0),2)

            dlib_box = dlib.rectangle(x, y, w, h)

            pose = face_landmarks_model(rgb, dlib_box)
            encoding = np.array(fr_model.compute_face_descriptor(rgb, pose, 1))

            name = svm.predict_proba([encoding])[0]
            max_arg = name.argmax()
            
            label = "Others"
            
            if name[max_arg] > 0.96:
                label = inv_label_dictionary[max_arg] + ": " + str(round(name[max_arg] * 100, 2))
                
            cv2.putText(rgb, label, (x-10,y-10), font, 0.6, (255,255,0), 1, cv2.LINE_AA)
        
    except Exception as e:
        print(e)
        pass
    
    img = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    cv2.imshow('img', img)
    
    if cv2.waitKey(1) == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

# Capture Image and Detect

In [4]:
cam = cv2.VideoCapture(0)
predictions = []
while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if cv2.waitKey(1) == ord('c'):
        try:
            faces = face_detector(gray, 1)
            for face_rect in faces:
                x, y, w, h = face_rect.left(), face_rect.top(), face_rect.right(), face_rect.bottom()
                cv2.rectangle(rgb,(x, y),(w, h),(0,255,0),2)
                dlib_box = dlib.rectangle(x, y, w, h)
                pose = face_landmarks_model(rgb, dlib_box)
                embedding = np.array(fr_model.compute_face_descriptor(rgb, pose, 1))
                name = svm.predict_proba([embedding])[0]

                max_arg = name.argmax()
                
                label = "Others"
                
                if name[max_arg] > 0.96:
                    predictions.append(inv_label_dictionary[max_arg])
                    label = inv_label_dictionary[max_arg] + ": " + str(round(name[max_arg] * 100, 2))
                else:
                    predictions.append("Others")
                
                cv2.putText(rgb, label, (x-10,y-10), font, 0.6, (255, 255, 0), 1, cv2.LINE_AA)
                                
            break

        except Exception as e:
            print(e)
            pass
        
    img = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    cv2.imshow('img', img)
    
    if cv2.waitKey(1) == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()


img = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
cv2.imshow('output', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

print(predictions)

['Prateek']
